In [ ]:
pip install opencv-python

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Tasks A and B

<!-- For this task I began by implementing the convolution filter for a grayscale image to get an idea for how to approach the task. To do this I began by reading the image using `plt.imread` function and then use `cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)` to create a grayscale version of the image.  -->

## Applying a Convolution Filter on a grayscale image

Here the ratios by which I multiply is based on this article: [NTSC Formula for Grayscale](https://support.ptc.com/help/mathcad/r10.0/en/index.html#page/PTC_Mathcad_Help/example_grayscale_and_color_in_images.html)

In [ ]:
def ICV_to_grayscale(image):
    w, h, c = image.shape
    grayimg = np.zeros((w,h))
    grayimg = (0.299*image[:,:,0] + 0.587*image[:,:,1] + 0.114*image[:,:,2])
    return grayimg

Here I divide each kernel by its sum to get the average intensity kernel for each size (3x3 to 7x7). 

In [ ]:
image = plt.imread("car-1.jpg")
img = ICV_to_grayscale(image)
## I have defined kernels of different sizes to experiment and see how they change the effect of the filter

kernel_3 = np.array([
    [1,1,1],
    [1,1,1],
    [1,1,1],
])
avg_kernel_3 = kernel_3/np.sum(kernel_3)

kernel_5 = np.array([
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1],
])
avg_kernel_5 = kernel_5/np.sum(kernel_5)

kernel_7 = np.array([
    [1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1],
])
avg_kernel_7 = kernel_7/np.sum(kernel_7)
print(avg_kernel_3, avg_kernel_5, avg_kernel_7)
plt.imshow(image)

In [ ]:
def ICV_convolution_filtering_grayscale(image, kernel):
    """
    Applies a convolution kernel on a greyscale image.

    Parameters:
    image -> 2D numpy array
    kernel -> 2D numpy array (usually with shapes (3,3), (5,5) or (7,7))

    Returns:
    2D numpy array (Image with filter applied to it)
    """
    width, height = image.shape[:2] # Get width and height from image shape
    new_image = np.zeros((width, height)) # create new image array
    k_w, k_h = kernel.shape[:2] # Get width and height of convolution kernel

    ## I pad the image with zeroes to half the size of the 
    ## kernel's width/height before applying the filter
    pad_width, pad_height = (k_w-1)//2, (k_h-1)//2 

    ## The padded image is slightly larger to allow the kernels's to affect all pixels in the image
    ## The image is padded at the top, bottom, left and right edges
    padded_image = np.zeros((width + 2 * pad_width, height + 2 * pad_height))

    ## Put the original image back into the larger padded array
    padded_image[pad_width:pad_width + width, pad_height:pad_height + height] = image

    ## When iterating over the image to ensure that the kernel is properly applied
    ## and so we don't get out of bounds errors, I iterate till width/height -pad_width/pad_height
    for i in range(width-pad_width):
        for j in range(height-pad_height):
            ## Apply the kernel on each pixel of the original image 
            ## and store the results in the new_image array
            new_image[i+1, j+1] = np.sum(kernel * padded_image[i:i+k_w, j:j+k_h])
    return new_image

In [ ]:
filtered_img_gray = ICV_convolution_filtering_grayscale(img, avg_kernel_3)
# print(filtered_img_gray, filtered_img_gray.shape)
plt.imshow(filtered_img_gray, cmap="gray")

## Different attempts at trying to implement the same for an RGB image

### Attempt 1:

This version works but suffers from the boundary problem. Each time a convolution is applied, the image shrinks by half the size of the kernel in its width and height. 

In [ ]:
def ICV_convolution_filtering_rgb(image, kernel):
    """
    Applies a convolution kernel on a rgb image. (This implementation suffers from the border problem)

    Parameters:
    image -> 2D numpy array
    kernel -> 2D numpy array (usually with shapes (3,3), (5,5) or (7,7))

    Returns:
    2D numpy array (Image with filter applied to it)
    """
    width, height, channels = image.shape # Get width and height from image shape
    k_w, k_h = kernel.shape[:2] # get kernel shape
    print(image.shape)
    new_image = np.zeros((width-k_w+1, height-k_h+1, channels))
    for c in range(channels):
        for i in range(width-k_w+1):
            for j in range(height-k_h+1):
                new_image[i, j, c] = np.sum(kernel * image[i:i+k_w, j:j+k_h, c], axis=(0,1))
    return new_image.astype(int)

In [ ]:
filtered_img = ICV_convolution_filtering_rgb(image, avg_kernel_3)
print(filtered_img, filtered_img.shape)
plt.imshow(filtered_img)

### Attempt 2 :

Here I try to pad the image, but I overflowed/underflowed the image channel buffers. The image produced from it is quite dark and thus to see the result I don't divide filtered array by the sum of the kernel to obtain a brighter image. The resulting image has an interesting effect where there are colors all over the place and its hard to properly distinguish the subject of the image.

In [ ]:
def ICV_convolution_filtering_rgb_2(image, kernel):
    """
    Applies a convolution kernel on a rgb image. (Failed attempt)

    Parameters:
    image -> 2D numpy array
    kernel -> 2D numpy array (usually with shapes (3,3), (5,5) or (7,7))

    Returns:
    2D numpy array (Image with filter applied to it)
    """ 
    width, height, channels = image.shape # Get width and height from image shape
    k_w, k_h = kernel.shape[:2] # Get kernel shape
    pad_width, pad_height = (k_w-1)//2, (k_h-1)//2
    
    padded_image = np.zeros((width + 2 * pad_width, height + 2 * pad_height, channels))
    padded_image[pad_width:pad_width + width, pad_height:pad_height + height, :] = image

    new_image = np.zeros_like(image)
    
    for c in range(channels):
        for i in range(width-pad_width):
            for j in range(height-pad_height):
                new_image[i, j, c] = np.sum(kernel * padded_image[i:i+k_w, j:j+k_h, c])
    
    return new_image.astype(int)

In [ ]:
# Because of the reason mentioned in the MD section I have chosen to use kernel_3 instead of avg_kernel_3 for this example
filtered_img_rgb_2 = ICV_convolution_filtering_rgb_2(image, kernel_3)
# print(filtered_img_rgb_2, filtered_img_rgb_2.shape)
plt.imshow(filtered_img_rgb_2)

### Attempt 3: Finally successful!

After 2 failed attempts, I finally managed to apply the filter while retaining the image's original dimension.

In [ ]:
def ICV_convolution_filtering_rgb_3(image, kernel):
    width, height, channels = image.shape
    k_w, k_h = kernel.shape[:2]

    pad_width = k_w//2
    pad_height = k_h//2
    padded_image = np.zeros((width + 2 * pad_width, height + 2*pad_height, channels), dtype=image.dtype)

    padded_image[ pad_width: pad_width + width,  pad_height:pad_height + height] = image
    
    # padding = 1
    # padded_image = np.pad(image, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
    # padded_width, padded_height = padded_image.shape[:2]
    new_image = np.zeros((width, height, channels))

    ## Till this point everything is the same as the previous implementations
    
    for c in range(channels):
        ## The size of the new image should be the same as the original and thus I iterate the same dimensions
        ## as the original image
        for i in range(width):
            for j in range(height):
                ## Here I take the sum across both rows and columns of the product
                new_image[i, j, c] = np.sum(kernel * padded_image[i:i+k_w, j:j+k_h, c], axis=(0,1))
    
    return new_image.astype(int)

In [ ]:
filtered_img_rgb = ICV_convolution_filtering_rgb_3(image, avg_kernel_3) 
print(filtered_img_rgb.shape) # ,filtered_img_rgb)
plt.imshow(filtered_img_rgb)

### Experiments with Kernels larger than 3x3

Here I tested how the different kernels behave in the event a larger kernel of size such as 5x5 or 7x7 is applied instead of the standard 3*3 kernel. Here I'm doing this with the function I created in my first attempt (convolution_filtering_rgb) to highlight how larger kernels would make the boundary problem significantly worse.

In [ ]:
filtered_img = ICV_convolution_filtering_rgb(image, avg_kernel_5)
print("Filtered Shape:", filtered_img.shape)
plt.imshow(filtered_img)

In [ ]:
filtered_img = ICV_convolution_filtering_rgb(image, avg_kernel_7)
print("Filtered Shape:", filtered_img.shape)
plt.imshow(filtered_img)

As can be seen in the 2 examples above applying a bigger filter just accelarates the results of the boundary problem, with the image losing between 5 and 7 pixels each time the filter is applied. This would be especially detrimental for images with smaller dimensions like the ones in Dataset A which have dimensions 256*256.

### Original Image

In [ ]:
plt.imshow(image)

### Averaged Image (Kernel (5x5))

In [ ]:
filtered_img = ICV_convolution_filtering_rgb(image, avg_kernel_5)
print("Filtered Shape:", filtered_img.shape)
plt.imshow(filtered_img)

# Task C

## Applying Gaussian and Laplace Filters

In [ ]:
# Gaussian Blur Filter Kernel
kernel_a = np.array([
    [1,2,1],
    [2,4,2],
    [1,2,1],
])


# Laplace Filter Kernel
kernel_b = np.array([
    [0,1,0],
    [1,-4,1],
    [0,1,0],
])


# I also have the Sobel filter Kernel which I experimented with
kernel_c = np.array([
    [1,0,-1],
    [1,0,-1],
    [1,0,-1],
])

In [ ]:
## Applying Gaussian Filter (A)
filtered_img_a = ICV_convolution_filtering_rgb_3(image, kernel_a)//np.sum(kernel_a)
print("Filtered Result Shape: ", filtered_img_a.shape)
plt.imshow(filtered_img_a)

## Conversion and Thresholding

This is a helper function for the Laplace filter in Task C. This helps the resulting image to have sharper edges and uses a thresolding method and converts all pixel values in the image above a certain threshold (here I set it to around 50% of max intensity) to 255 and everything below it to 0. This also helps convert images with invalid pixel values (> 255 or < 0) to something which is still within the acceptable range. Some filters such as the laplace filter, when applied result in filters which create pixel values greater than 255 or negative pixel values. This can't directly be displayed so the image results need to converted for the best results. Jupyter Hub handles this as well, but its good practice to remove unexpected behaviour and keep the pixel values within the acceptable range

In [ ]:
def ICV_clip_image_rgb(image):
    width, height, channels = image.shape
    threshold = 122 # 50 % of max intensity (255)
    for c in range(channels):
        for i in range(width):
            for j in range(width):
                if image[i][j][c] < 0:
                    image[i][j][c] = 0
                elif image[i][j][c] > threshold: 
                    image[i][j][c] = 255
    return image

In [ ]:
## Applying Laplace Filter (B)
filtered_img_b = ICV_convolution_filtering_rgb_3(image, kernel_b)
print("Filtered Result Shape:", filtered_img_b.shape)
## Here I print the max and min pixel value to show why we need to clip the filtered image 
print("Max filtered pixel value: ", np.amax(filtered_img_b), " | Min filtered pixel value: ", np.amin(filtered_img_b))
filtered_img_b = ICV_clip_image_rgb(filtered_img_b)
plt.imshow(filtered_img_b)

# Task D

### Subtask 1

For task C I had applied the filter A on the image already so use its result here directly (filtered_img_a)

In [ ]:
filtered_img_a_a = ICV_convolution_filtering_rgb_3(filtered_img_a, kernel_a)//np.sum(kernel_a) 
print("Filtered Result Shape:", filtered_img_a_a.shape)
plt.imshow(filtered_img_a_a, cmap="gray")

### Subtask 2 ((Applying Filter A followed Filter B))
Same as in the previous cell, I use the result of applying kernel_a and kernel_b on the image here directly (filtered_img_a and filtered_image_b).

In [ ]:
filtered_img_a_b = ICV_convolution_filtering_rgb_3(filtered_img_a, kernel_b)
print("Filtered Result Shape:",  filtered_img_a_b.shape)
filtered_img_a_b = ICV_clip_image_rgb(filtered_img_a_b)
plt.imshow(filtered_img_a_b)

### Subtask 3 (Applying Filter B followed Filter A)

In [ ]:
filtered_img_b_a = ICV_convolution_filtering_rgb_3(filtered_img_b, kernel_a)//np.sum(kernel_a)
print("Filtered Result Shape:",  filtered_img_b_a.shape)
plt.imshow(filtered_img_b_a)